In [1]:
# aid.ipynb
# Author: Kyle Larson
# Purpose top secret
import numpy as np 
import soundfile as sf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Stack overflow was consulted in order to parse '.flac' 
#path = '../input/rfcx-species-audio-detection/train/00204008d.flac'
#data, samplerate = sf.read(path) #this method is thanks to Harry Moreno 
#from his Stack Overflow response on June 13 '18 at the following URL:
#https://stackoverflow.com/questions/50804170/load-flac-file-in-python-same-as-scipy-or-librosa


In [2]:
import os
import pandas as pd
file_name_list = os.listdir('C:/Users/admin/anaconda3/01 PROJECTS/Audio ID')
#the following sequential open strategy was found not to work due to frequency mismatch between id_data and filenames
ignore = ['.git', '.ipynb_checkpoints', 'train.zip']
new_df1 = pd.DataFrame({file_name_list[2]:[]})
for i in range(len(file_name_list)):#len(id_data) #10 files ~ 5 seconds w/ RAM ~25%
    if(file_name_list[i]!=ignore[0] and file_name_list[i]!=ignore[1] and file_name_list[i]!=ignore[2]):
        data, samplerate = sf.read('C:/Users/admin/anaconda3/01 PROJECTS/Audio ID/' +file_name_list[i])
        new_df1[file_name_list[i]] = np.array(data)
    if(i==10):
        break
new_df1.head() 

,00204008d.flac,003b04435.flac,003bec244.flac,005f1f9a5.flac,006ab765f.flac,0072f0839.flac,0079ff47b.flac,007f87ba2.flac,00834f88e.flac
0,0.007538,0.002106,0.008301,0.015076,-0.009308,-0.001404,-0.028168,0.047791,-0.067291
1,0.007111,0.007935,0.028137,0.026093,-0.009949,-0.000092,0.016174,0.030731,-0.066895
2,0.006165,0.005188,0.024689,0.029877,-0.011444,0.003693,0.070160,0.016846,-0.059387
3,0.010773,0.006744,0.019836,0.038025,-0.010559,-0.004700,0.078705,-0.004059,-0.030823
4,0.004669,0.004028,-0.002319,0.042023,-0.011383,0.002777,0.102417,-0.026215,0.000427


In [3]:
#at 1 min 5 sec the following error occured:
#Your notebook tried to allocate more memory than is available. It has restarted.
#thats 65 seconds or 130 files that can occupy ram at once

In [4]:
len(new_df1.loc[0])

9

In [5]:
#the plan is to use a fft (Fast Forier Transform) to halve the number of datapoints to reduce compute time
#then use an SVD (singular value decomposition) to prioritize the points that are principle components
from scipy.fft import fft, ifft
fft_aid =[]
for i in range(2, len(new_df1.loc[0])):#2 is to avoid auto '.git' undesirables, depends on intividual system
    x = np.array(new_df1.loc[:][file_name_list[i]])
    fft_aid += [fft(x)]
print(fft_aid)
    

#time for this 9 was <1sec

[array([41.13421631-0.j        , 10.64642932+0.25820614j,
        9.2332184 -0.19021899j, ...,  7.33564754-1.87112699j,
        9.2332184 +0.19021899j, 10.64642932-0.25820614j]), array([40.94290161 -0.j        , -7.09001743+15.60606578j,
       -3.8866546  +6.41422956j, ...,  0.11746973 -7.08458867j,
       -3.8866546  -6.41422956j, -7.09001743-15.60606578j]), array([73.42666626 -0.j        , -7.19791793+10.73032501j,
        6.07912089+16.33527279j, ..., 10.57079271 -7.26532391j,
        6.07912089-16.33527279j, -7.19791793-10.73032501j]), array([ 338.96380615 -0.j        , -195.86090338+22.05340036j,
        234.4222708 -23.29121672j, ..., -221.97790813-58.74850902j,
        234.4222708 +23.29121672j, -195.86090338-22.05340036j]), array([12.26766968-0.j        , -0.78639117-5.08030292j,
        5.61737918+2.27025603j, ..., 10.45377886+5.74960348j,
        5.61737918-2.27025603j, -0.78639117+5.08030292j]), array([-23.9730835 -0.j        ,  14.18005352-0.28015113j,
        16.20927771-

In [6]:
import numpy as np
def drop_second_half(array):
    array = array[0:int(np.ceil((len(array)/2)))]
    return array

test=[1,2,3,4,5,6,7]

print(drop_second_half(test))

[1, 2, 3, 4]


In [7]:
half_fft_aid = []
for i in range(len(fft_aid)):
    #print(fft_aid[i])
    print(len(fft_aid[i][:]))
    half_fft_aid += [drop_second_half(fft_aid[i][:])]
    print(len(half_fft_aid[i]))

2880000
1440000
2880000
1440000
2880000
1440000
2880000
1440000
2880000
1440000
2880000
1440000
2880000
1440000


In [8]:
from numpy import array
from scipy.linalg import svd
from numpy import dot
from numpy import diag
from numpy import zeros

print(half_fft_aid)
U, s, VT = svd(half_fft_aid)
Sigma = zeros(half_fft_aid.shape[0], half_fft_aid.shape[1])
Sigma[:half_fft_aid.shape[1], :half_fft_aid.shape[1]] = diag(s)
B = U.dot(Sigma.dot(VT))
print(B)

[array([41.13421631-0.j        , 10.64642932+0.25820614j,
        9.2332184 -0.19021899j, ..., -2.39748186+1.01769676j,
       -1.65670837+0.16250497j,  3.64337491-3.08708386j]), array([40.94290161 -0.j        , -7.09001743+15.60606578j,
       -3.8866546  +6.41422956j, ...,  0.70134716 -3.96830537j,
       -2.47606737 +0.5286674j , -4.86818625 -7.61115003j]), array([73.42666626 -0.j        , -7.19791793+10.73032501j,
        6.07912089+16.33527279j, ..., -0.29435009 +0.4838166j ,
       -0.87194791 +1.88824643j,  5.66523017 +4.93267021j]), array([ 338.96380615 -0.j        , -195.86090338+22.05340036j,
        234.4222708 -23.29121672j, ...,    8.65328552 -5.23795023j,
        -10.69537989 +1.52445274j,    2.76814016 +0.03399624j]), array([12.26766968-0.j        , -0.78639117-5.08030292j,
        5.61737918+2.27025603j, ...,  3.11120493+0.98640752j,
        2.38571514+0.80971595j, -0.43046878+0.86299398j]), array([-23.9730835 -0.j        ,  14.18005352-0.28015113j,
        16.20927771-

MemoryError: Unable to allocate 30.2 TiB for an array with shape (1440000, 1440000) and data type complex128